# Get config.h log from buildbot given the build url

In [1]:
builder_url = "https://www.octopus-code.org/buildbot/#/builders/144/builds/194"
config_page = f"{builder_url}/steps/5/logs/config_h"

In [1]:
import requests

In [3]:
response = requests.get(
    'https://www.octopus-code.org/buildbot/api/v2/builders/144/builds/194',
)

In [4]:
response.content;

In [5]:
response = requests.get('https://www.octopus-code.org/buildbot/api/v2/builds/61409/steps/5') # buildid from above ;


In [6]:
response.content;

In [7]:
response = requests.get(
    'https://www.octopus-code.org/buildbot/api/v2/steps/568186/logs/config_h',
)
# stepid from above

In [8]:
response.content;

In [9]:
response = requests.get(
    'https://www.octopus-code.org/buildbot/api/v2/logs/709753/contents',
)
# logid from above

In [10]:
response.content;

# MVP function

In [2]:
import json
import requests
import re

In [3]:
def get_config_h(build_url:str)->str:
    """
    Return the contents of the config.h file from the buildbot

    Assuming build_url is of the form:
    builder_url = "https://www.octopus-code.org/buildbot/#/builders/144/builds/194"
    return the contents of the config.h file
    by first finding the buildid then the stepid then the logid
    """
    # get builders_id and builds_id
    # match for the pattern: builders/144 from that get the 144 for builders_id
    builders_id = re.search(r'builders/(\d+)', build_url).group(1)
    # match for the pattern: builds/194 from that get the 194 for builds_id
    builds_id = re.search(r'builds/(\d+)', build_url).group(1)
    # get the buildbot_website_url from the build_url for eg www.octopus-code.org
    buildbot_website_url = re.search(r'https?:\/\/([^\/#?]+)', build_url).group(1)

    # get buildid
    buildid_reponse = requests.get(f"http://{buildbot_website_url}/buildbot/api/v2/builders/{builders_id}/builds/{builds_id}")
    buildid = json.loads(buildid_reponse.content)['builds'][0]['buildid']  # buildid = 61409

    # get stepid  from 'https://www.octopus-code.org/buildbot/api/v2/builds/61409/steps/5'
    stepid_response = requests.get(f"http://{buildbot_website_url}/buildbot/api/v2/builds/{buildid}/steps/5")
    stepid = json.loads(stepid_response.content)['steps'][0]['stepid'] # stepid = 568186

    # get logid from https://www.octopus-code.org/buildbot/api/v2/steps/568186/logs/config_h
    logid_response = requests.get(f"http://{buildbot_website_url}/buildbot/api/v2/steps/{stepid}/logs/config_h")
    logid = json.loads(logid_response.content)['logs'][0]['logid'] # logid = 709753

    # get config.h from 'https://www.octopus-code.org/buildbot/api/v2/logs/709753/contents'
    config_h_response = requests.get(f"http://{buildbot_website_url}/buildbot/api/v2/logs/{logid}/contents")
    config_h = json.loads(config_h_response.content)["logchunks"][0]["content"]
    return config_h.replace('o\n','\n').replace('\no','\n')


In [4]:
sample_config_h = get_config_h( "https://www.octopus-code.org/buildbot/#/builders/144/builds/194")

In [5]:
sample_config_h;

In [6]:
# print(get_config_h("https://www.octopus-code.org/buildbot/#/builders/144/builds/192"))

# Get a diff

In [7]:
import difflib

from rich.markdown import Markdown
from rich import print as rprint
import inspect

In [8]:
f1_content = get_config_h("https://www.octopus-code.org/buildbot/#/builders/144/builds/194")
f2_content = get_config_h("https://www.octopus-code.org/buildbot/#/builders/144/builds/192")

In [9]:
diff = difflib.unified_diff(
                f1_content.split("\n"),
                f2_content.split("\n"),
                "left",
                "right",
            )
diffy = "\n".join(list(diff))

In [10]:
diffy_body =Markdown(
f"""
```diff
{diffy}
```
""",
code_theme="vim",
)

In [11]:
rprint(diffy_body);

                                                                                                                   
 --- left                                                                                                          
                                                                                                                   
 +++ right                                                                                                         
                                                                                                                   
 @@ -2,7 +2,7 @@                                                                                                   
                                                                                                                   
  /* config.h.in.  Generated from configure.ac by autoheader.  */                                                  
                                                                                                                   
  /* date when configure was launched */                                                                           
 -#define BUILD_TIME "Wed Dec  6 10:51:00 CET 2023"                                                                
 +#define BUILD_TIME "Wed Dec  6 09:29:51 CET 2023"                                                                
                                                                                                                   
  /* C compiler */                                                                                                 
  #define CC "gcc "                                                                                                
 @@ -73,7 +73,7 @@                                                                                                 
                                                                                                                   
  #define FC_INTEGER_SIZE 4                                                                                        
                                                                                                                   
  /* git commit hash */                                                                                            
 -#define GIT_COMMIT "2db69a4030c6cf72ab92cfc37b31671a23b2541f"                                                    
 +#define GIT_COMMIT "01319e380b0e8f85703bf2d9c6652877a52afa53"                                                    
                                                                                                                   
  /* Define to 1 if you have 'alloca', as a function or macro. */                                                  
  #define HAVE_ALLOCA 1                                                                                            
                                                                                                                   

In [12]:
def get_diff(str_old:str, str_new:str,str_old_name="old",str_new_name="new"):
    diff = difflib.unified_diff(
                str_old.split("\n"),
                str_new.split("\n"),
                str_old_name,
                str_new_name
            )
    diffy = "\n".join(list(diff))

    diffy_body = Markdown(
    f"""
```diff
{diffy}
```
    """,
    code_theme="vim",
    )

    return diffy_body

In [22]:
get_diff(f1_content,f2_content)

                                                                                                                   
 --- old                                                                                                           
                                                                                                                   
 +++ new                                                                                                           
                                                                                                                   
 @@ -2,7 +2,7 @@                                                                                                   
                                                                                                                   
  /* config.h.in.  Generated from configure.ac by autoheader.  */                                                  
                                                                                                                   
  /* date when configure was launched */                                                                           
 -#define BUILD_TIME "Wed Dec  6 10:51:00 CET 2023"                                                                
 +#define BUILD_TIME "Wed Dec  6 09:29:51 CET 2023"                                                                
                                                                                                                   
  /* C compiler */                                                                                                 
  #define CC "gcc "                                                                                                
 @@ -73,7 +73,7 @@                                                                                                 
                                                                                                                   
  #define FC_INTEGER_SIZE 4                                                                                        
                                                                                                                   
  /* git commit hash */                                                                                            
 -#define GIT_COMMIT "2db69a4030c6cf72ab92cfc37b31671a23b2541f"                                                    
 +#define GIT_COMMIT "01319e380b0e8f85703bf2d9c6652877a52afa53"                                                    
                                                                                                                   
  /* Define to 1 if you have 'alloca', as a function or macro. */                                                  
  #define HAVE_ALLOCA 1                                                                                            
                                                                                                                   


# Diff eb(left) vs spack(right)

In [23]:
# Builder map for branch "spack-builder-tollerances-round2" revision "2552abad9f91a5d93a046214bb6520d44507a8f1"
# test suite app https://octopus-code.org/testsuite/commits/1636
# eb pipeline https://gitlab.com/octopus-code/octopus/-/pipelines/1094456375
# spack pipeline https://www.octopus-code.org/buildbot/#/builders/137/builds/17

# builder_map = {builder:{easybuild:eb_url,spack:spack_url}}
example_map = {
    "foss2022a_serial_min": {
        "easybuild": "_",
        "spack": "https://www.octopus-code.org/buildbot/#/builders/144/builds/155",
    },
    "foss2022a_serial": {
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/51/builds/1775",
        "spack": "https://www.octopus-code.org/buildbot/#/builders/100/builds/35",
    },
    "foss2022a_serial_omp": {
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/31/builds/950",
        "spack": "https://www.octopus-code.org/buildbot/#/builders/139/builds/20",
    },
    "foss2022a_serial_debug": {
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/37/builds/1756",
        "spack": "https://www.octopus-code.org/buildbot/#/builders/151/builds/10",
    },
    "foss2022a_mpi": {
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/41/builds/1802",
        "spack": "https://www.octopus-code.org/buildbot/#/builders/102/builds/34",
    },
    "foss2022a_mpi_omp": {
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/53/builds/970",
        "spack": "https://www.octopus-code.org/buildbot/#/builders/138/builds/17",
    },
    "foss2022a_mpi_debug": {
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/25/builds/44",
        "spack": "https://www.octopus-code.org/buildbot/#/builders/152/builds/11",
    },
    "foss2022a_cuda_mpi_omp": {
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/147/builds/20",
        "spack": "https://www.octopus-code.org/buildbot/#/builders/159/builds/5",
    },
}


In [24]:
for builder in example_map:
    left_url = example_map[builder]["easybuild"]
    right_url = example_map[builder]["spack"]
    headding = Markdown(f"""
## {builder=}
### {left_url=}
### {right_url=}
""")
    rprint(headding)
    try:
        left_config_h = get_config_h(left_url)
        right_config_h = get_config_h(right_url)
        diff = get_diff(
            left_config_h, right_config_h, f"eb_{builder}", f"spack_{builder}"
        )
        rprint(diff)
    except:
        rprint(Markdown("### failed to compile results"))

builder='foss2022a_serial_min'                                           

                                                   left_url='_'                                                    

                    right_url='https://www.octopus-code.org/buildbot/#/builders/144/builds/155'

failed to compile results

builder='foss2022a_serial'                                             

                    left_url='https://www.octopus-code.org/buildbot/#/builders/51/builds/1775'                     

                    right_url='https://www.octopus-code.org/buildbot/#/builders/100/builds/35'

                                                                                                                   
 --- eb_foss2022a_serial                                                                                           
                                                                                                                   
 +++ spack_foss2022a_serial                                                                                        
                                                                                                                   
 @@ -2,7 +2,7 @@                                                                                                   
                                                                                                                   
  /* config.h.in.  Generated from configure.ac by autoheader.  */                                                  
                                                                                                                   
  /* date when configure was launched */                                                                           
 -#define BUILD_TIME "Tue Dec  5 22:39:14 CET 2023"                                                                
 +#define BUILD_TIME "Mon Dec  4 14:17:04 CET 2023"                                                                
                                                                                                                   
  /* C compiler */                                                                                                 
  #define CC "gcc "                                                                                                
 @@ -35,7 +35,7 @@                                                                                                 
                                                                                                                   
  #define FC "gfortran "                                                                                           
                                                                                                                   
  /* Fortran compiler flags */                                                                                     
 -#define FCFLAGS "-Wall -Wno-maybe-uninitialized -Wno-surprising -O2 -march=native -fbacktrace -ftest-coverage    
 -fprofile-arcs"                                                                                                   
 +#define FCFLAGS "-Wall -Wno-maybe-uninitialized -Wno-surprising -O2 -march=native -fbacktrace  -ftest-coverage   
 -fprofile-arcs"                                                                                                   
                                                                                                                   
  /* Fortran compiler flags (extra) */                                                                             
  #define FCFLAGS_EXTRA ""                                                                                         
 @@ -127,7 +127,7 @@                                                                                               
                                                                                                                   
  /* #undef HAVE_CL_CL_H */                                                                                        
                                                                                                                   
  /* This is defined when we link with an external ISF library. */                                                 
 -/* #undef HAVE_COMP_ISF */                                                                                       
 +#define HAVE_COMP_ISF 1                                                                                          
                                                        

builder='foss2022a_serial_omp'                                           

                     left_url='https://www.octopus-code.org/buildbot/#/builders/31/builds/950'                     

                    right_url='https://www.octopus-code.org/buildbot/#/builders/139/builds/20'

...

                                                                                                                   
 --- eb_foss2022a_mpi_omp                                                                                          
                                                                                                                   
 +++ spack_foss2022a_mpi_omp                                                                                       
                                                                                                                   
 @@ -2,7 +2,7 @@                                                                                                   
                                                                                                                   
  /* config.h.in.  Generated from configure.ac by autoheader.  */                                                  
                                                                                                                   
  /* date when configure was launched */                                                                           
 -#define BUILD_TIME "Tue Dec  5 22:39:22 CET 2023"                                                                
 +#define BUILD_TIME "Mon Dec  4 14:17:09 CET 2023"                                                                
                                                                                                                   
  /* C compiler */                                                                                                 
  #define CC "mpicc (gcc)"                                                                                         
 @@ -32,7 +32,7 @@                                                                                                 
                                                                                                                   
  #define F90_ACCEPTS_LINE_NUMBERS 1                                                                               
                                                                                                                   
  /* Fortran compiler */                                                                                           
 -#define FC "mpifort (gfortran)"                                                                                  
 +#define FC "mpif90 (gfortran)"                                                                                   
                                                                                                                   
  /* Fortran compiler flags */                                                                                     
  #define FCFLAGS "-Wall -Wno-maybe-uninitialized -Wno-surprising -O3 -march=native -fbacktrace -funroll-loops     
 -fopenmp"                                                                                                         
 @@ -85,7 +85,7 @@                                                                                                 
                                                                                                                   
  #define HAVE_ALPHASORT 1                                                                                         
                                                                                                                   
  /* Defined if you have the bigdft-atlab library. */                                                              
 -/* #undef HAVE_ATLAB */                                                                                          
 +#define HAVE_ATLAB 1                                                                                             
                                                                                                                   
  /* Defined if you have the BerkeleyGW library. */     

builder='foss2022a_mpi_debug'                                           

                     left_url='https://www.octopus-code.org/buildbot/#/builders/25/builds/44'                      

                    right_url='https://www.octopus-code.org/buildbot/#/builders/152/builds/11'

                                                                                                                   
 --- eb_foss2022a_mpi_debug                                                                                        
                                                                                                                   
 +++ spack_foss2022a_mpi_debug                                                                                     
                                                                                                                   
 @@ -2,7 +2,7 @@                                                                                                   
                                                                                                                   
  /* config.h.in.  Generated from configure.ac by autoheader.  */                                                  
                                                                                                                   
  /* date when configure was launched */                                                                           
 -#define BUILD_TIME "Tue Dec  5 22:40:00 CET 2023"                                                                
 +#define BUILD_TIME "Mon Dec  4 14:18:00 CET 2023"                                                                
                                                                                                                   
  /* C compiler */                                                                                                 
  #define CC "mpicc (gcc)"                                                                                         
 @@ -32,13 +32,13 @@                                                                                               
                                                                                                                   
  #define F90_ACCEPTS_LINE_NUMBERS 1                                                                               
                                                                                                                   
  /* Fortran compiler */                                                                                           
 -#define FC "mpifort (gfortran)"                                                                                  
 +#define FC "mpif90 (gfortran)"                                                                                   
                                                                                                                   
  /* Fortran compiler flags */                                                                                     
  #define FCFLAGS "-g -fno-var-tracking-assignments -Wall -Wno-maybe-uninitialized -Wno-surprising -O2 -march=nati 
 -fbacktrace -fcheck=al"                                                                                           
                                                                                                                   
  /* Fortran compiler flags (extra) */                                                                             
 -#define FCFLAGS_EXTRA "l -fbounds-check -finit-real=snan -ffpe-trap=zero,invalid -ftest-coverage -fprofile-arcs" 
 +#define FCFLAGS_EXTRA "l -fbounds-check -finit-real=snan -finit-logical=false -finit-derived                     
 -ffpe-trap=zero,invalid -ftest-coverage -fprofile-"                                                               
                                                                                                                   
  /* compiler supports command line arguments */                                                                   
  #define FC_COMMAND_LINE_ARGUMENTS 2003                                                                           
 @@ -88,10 +88,10 @@                                    

builder='foss2022a_cuda_mpi_omp'                                          

                     left_url='https://www.octopus-code.org/buildbot/#/builders/147/builds/20'                     

                     right_url='https://www.octopus-code.org/buildbot/#/builders/159/builds/5'

                                                                                                                   
 --- eb_foss2022a_cuda_mpi_omp                                                                                     
                                                                                                                   
 +++ spack_foss2022a_cuda_mpi_omp                                                                                  
                                                                                                                   
 @@ -2,7 +2,7 @@                                                                                                   
                                                                                                                   
  /* config.h.in.  Generated from configure.ac by autoheader.  */                                                  
                                                                                                                   
  /* date when configure was launched */                                                                           
 -#define BUILD_TIME "Mon Dec  4 15:44:43 CET 2023"                                                                
 +#define BUILD_TIME "Mon Dec  4 14:56:11 CET 2023"                                                                
                                                                                                                   
  /* C compiler */                                                                                                 
  #define CC "mpicc (gcc)"                                                                                         
 @@ -11,7 +11,7 @@                                                                                                 
                                                                                                                   
  #define CC_FORTRAN_INT int                                                                                       
                                                                                                                   
  /* C compiler flags */                                                                                           
 -#define CFLAGS "-Wall -O3 -march=native -funroll-loops"                                                          
 +#define CFLAGS "-Wall -O2 -march=native"                                                                         
                                                                                                                   
  /* C compiler flags (extra) */                                                                                   
  #define CFLAGS_EXTRA ""                                                                                          
 @@ -32,13 +32,13 @@                                                                                               
                                                                                                                   
  #define F90_ACCEPTS_LINE_NUMBERS 1                                                                               
                                                                                                                   
  /* Fortran compiler */                                                                                           
 -#define FC "mpifort (gfortran)"                                                                                  
 +#define FC "mpif90 (gfortran)"                                                                                   
                                                                                                                   
  /* Fortran compiler flags */                                                                                     
 -#define FCFLAGS "-Wall -Wno-maybe-uninitialized -Wno-s

# Single example

In [25]:
example_map = {
    "foss2022a_mpi_debug": {
        "new": "http://mpsd-octopus-code-staging.desy.de/buildbot/#/builders/158/builds/7",
        "old": "http://mpsd-octopus-code-staging.desy.de/buildbot/#/builders/158/builds/5",
    },
}

for builder in example_map:
    [left,right] = list(example_map[builder].keys())
    left_url = example_map[builder][left]
    right_url = example_map[builder][right]
    headding = Markdown(f"""
## {builder=}
### {left_url=}
### {right_url=}
""")
    rprint(headding)
    try:
        left_config_h = get_config_h(left_url)
        right_config_h = get_config_h(right_url)
        diff = get_diff(
            left_config_h, right_config_h, left, right
        )
        rprint(diff)
    except:
        rprint(Markdown("### failed to compile results"))


builder='foss2022a_mpi_debug'                                           

               left_url='http://mpsd-octopus-code-staging.desy.de/buildbot/#/builders/158/builds/7'                

               right_url='http://mpsd-octopus-code-staging.desy.de/buildbot/#/builders/158/builds/5'

                                                                                                                   
 --- new                                                                                                           
                                                                                                                   
 +++ old                                                                                                           
                                                                                                                   
 @@ -2,7 +2,7 @@                                                                                                   
                                                                                                                   
  /* config.h.in.  Generated from configure.ac by autoheader.  */                                                  
                                                                                                                   
  /* date when configure was launched */                                                                           
 -#define BUILD_TIME "Thu Dec  7 15:51:22 CET 2023"                                                                
 +#define BUILD_TIME "Wed Nov 29 19:02:47 CET 2023"                                                                
                                                                                                                   
  /* C compiler */                                                                                                 
  #define CC "mpicc (gcc)"                                                                                         
 @@ -73,7 +73,7 @@                                                                                                 
                                                                                                                   
  #define FC_INTEGER_SIZE 4                                                                                        
                                                                                                                   
  /* git commit hash */                                                                                            
 -#define GIT_COMMIT "b69e5157fdbc64ecc388d582b1c998b07e546f46"                                                    
 +#define GIT_COMMIT "cc8d49a2f34e33c6213ca03ea5739f5beeaa8c78"                                                    
                                                                                                                   
  /* Define to 1 if you have 'alloca', as a function or macro. */                                                  
  #define HAVE_ALLOCA 1                                                                                            
                                                                                                                   

In [29]:
example_map = {
    "foss2022a_serial_full": {
        "spack": "http://mpsd-octopus-code-staging.desy.de/buildbot/#/builders/99/builds/10",
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/51/builds/1775",
    },
}

for builder in example_map:
    [left,right] = list(example_map[builder].keys())
    left_url = example_map[builder][left]
    right_url = example_map[builder][right]
    headding = Markdown(f"""
## {builder=}
### {left_url=}
### {right_url=}
""")
    rprint(headding)
    try:
        left_config_h = get_config_h(left_url)
        right_config_h = get_config_h(right_url)
        diff = get_diff(
            left_config_h, right_config_h, left, right
        )
        rprint(diff)
    except:
        rprint(Markdown("### failed to compile results"))


builder='foss2022a_serial_full'                                          

               left_url='http://mpsd-octopus-code-staging.desy.de/buildbot/#/builders/99/builds/10'                

                    right_url='https://www.octopus-code.org/buildbot/#/builders/51/builds/1775'

                                                                                                                   
 --- spack                                                                                                         
                                                                                                                   
 +++ easybuild                                                                                                     
                                                                                                                   
 @@ -2,7 +2,7 @@                                                                                                   
                                                                                                                   
  /* config.h.in.  Generated from configure.ac by autoheader.  */                                                  
                                                                                                                   
  /* date when configure was launched */                                                                           
 -#define BUILD_TIME "Mon Dec 11 17:49:32 CET 2023"                                                                
 +#define BUILD_TIME "Tue Dec  5 22:39:14 CET 2023"                                                                
                                                                                                                   
  /* C compiler */                                                                                                 
  #define CC "gcc "                                                                                                
 @@ -73,7 +73,7 @@                                                                                                 
                                                                                                                   
  #define FC_INTEGER_SIZE 4                                                                                        
                                                                                                                   
  /* git commit hash */                                                                                            
 -#define GIT_COMMIT "b3bdfa75a94d46f12f670090719ec1b5f16d245a"                                                    
 +#define GIT_COMMIT "2552abad9f91a5d93a046214bb6520d44507a8f1"                                                    
                                                                                                                   
  /* Define to 1 if you have 'alloca', as a function or macro. */                                                  
  #define HAVE_ALLOCA 1                                                                                            
 @@ -85,7 +85,7 @@                                                                                                 
                                                                                                                   
  #define HAVE_ALPHASORT 1                                                                                         
                                                                                                                   
  /* Defined if you have the bigdft-atlab library. */                                                              
 -#define HAVE_ATLAB 1                                                                                             
 +/* #undef HAVE_ATLAB */                                                                                          
                                                                                                                   
  /* Defined if you have the BerkeleyGW library. */                                                                
  #define HAVE_BERKELEYGW 1                             

# latest example

In [27]:
# Builder map for branch "main" revision "ec425e2f042df5d6311b11ddbb7a48b32b47d10f"
# test suite app https://octopus-code.org/testsuite/commits/1703
# eb pipeline https://gitlab.com/octopus-code/octopus/-/pipelines/1094456375
# spack pipeline https://www.octopus-code.org/buildbot/#/builders/137/builds/17

# builder_map = {builder:{easybuild:eb_url,spack:spack_url}}
latest_example_map = {
    "foss2022a_serial_min": {
        "easybuild": "_",
        "spack": "https://www.octopus-code.org/buildbot/#/builders/144/builds/155",
    },
    "foss2022a_serial": {
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/51/builds/1775",
        "spack": "https://www.octopus-code.org/buildbot/#/builders/100/builds/35",
    },
    "foss2022a_serial_omp": {
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/31/builds/950",
        "spack": "https://www.octopus-code.org/buildbot/#/builders/139/builds/20",
    },
    "foss2022a_serial_debug": {
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/37/builds/1756",
        "spack": "https://www.octopus-code.org/buildbot/#/builders/151/builds/10",
    },
    "foss2022a_mpi": {
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/41/builds/1802",
        "spack": "https://www.octopus-code.org/buildbot/#/builders/102/builds/34",
    },
    "foss2022a_mpi_omp": {
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/53/builds/970",
        "spack": "https://www.octopus-code.org/buildbot/#/builders/138/builds/17",
    },
    "foss2022a_mpi_debug": {
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/25/builds/44",
        "spack": "https://www.octopus-code.org/buildbot/#/builders/152/builds/11",
    },
    "foss2022a_cuda_mpi_omp": {
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/147/builds/20",
        "spack": "https://www.octopus-code.org/buildbot/#/builders/159/builds/5",
    },
}


In [28]:
for builder in latest_example_map:
    left_url = example_map[builder]["easybuild"]
    right_url = example_map[builder]["spack"]
    headding = Markdown(f"""
## {builder=}
### {left_url=}
### {right_url=}
""")
    rprint(headding)
    try:
        left_config_h = get_config_h(left_url)
        right_config_h = get_config_h(right_url)
        diff = get_diff(
            left_config_h, right_config_h, f"eb_{builder}", f"spack_{builder}"
        )
        rprint(diff)
    except:
        rprint(Markdown("### failed to compile results"))

KeyError: 'foss2022a_serial_min'

# All serial examples from latest main 12/12/23

In [13]:
# Builder map for branch "main" revision "b3bdfa75a94d46f12f670090719ec1b5f16d245a"
# test suite app https://octopus-code.org/testsuite/commits/1733
# eb pipeline https://www.octopus-code.org/buildbot/#/builders/50/builds/2253
# spack pipeline https://www.octopus-code.org/buildbot/#/builders/137/builds/21

# builder_map = {builder:{easybuild:eb_url,spack:spack_url}}
latest_example_map = {
    "foss2022a_serial_min": {
        "easybuild": "_",
        "spack": "http://mpsd-octopus-code-staging.desy.de/buildbot/#/builders/150/builds/5",
    },
    "foss2022a_serial": {
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/110/builds/15",
        # "easybuild": "https://www.octopus-code.org/buildbot/#/builders/51/builds/1775",
        "spack": "http://mpsd-octopus-code-staging.desy.de/buildbot/#/builders/99/builds/11",
    },
    "foss2022a_serial_omp": {
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/31/builds/1024",
        "spack": "http://mpsd-octopus-code-staging.desy.de/buildbot/#/builders/141/builds/7",
    },
    "foss2022a_serial_debug": {
        "easybuild": "https://www.octopus-code.org/buildbot/#/builders/37/builds/1831",
        "spack": "http://mpsd-octopus-code-staging.desy.de/buildbot/#/builders/152/builds/10",
    },
}


In [14]:
for builder in latest_example_map:
    left_url = latest_example_map[builder]["easybuild"]
    right_url = latest_example_map[builder]["spack"]
    headding = Markdown(f"""
## {builder=}
### {left_url=}
### {right_url=}
""")
    rprint(headding)
    try:
        left_config_h = get_config_h(left_url)
        right_config_h = get_config_h(right_url)
        diff = get_diff(
            left_config_h, right_config_h, f"eb_{builder}", f"spack_{builder}"
        )
        rprint(diff)
    except:
        rprint(Markdown("### failed to compile results"))

builder='foss2022a_serial_min'                                           

                                                   left_url='_'                                                    

               right_url='http://mpsd-octopus-code-staging.desy.de/buildbot/#/builders/150/builds/5'

failed to compile results

builder='foss2022a_serial'                                             

                     left_url='https://www.octopus-code.org/buildbot/#/builders/110/builds/15'                     

               right_url='http://mpsd-octopus-code-staging.desy.de/buildbot/#/builders/99/builds/11'

                                                                                                                   
 --- eb_foss2022a_serial                                                                                           
                                                                                                                   
 +++ spack_foss2022a_serial                                                                                        
                                                                                                                   
 @@ -2,7 +2,7 @@                                                                                                   
                                                                                                                   
  /* config.h.in.  Generated from configure.ac by autoheader.  */                                                  
                                                                                                                   
  /* date when configure was launched */                                                                           
 -#define BUILD_TIME "Mon Dec 11 22:11:00 CET 2023"                                                                
 +#define BUILD_TIME "Tue Dec 12 11:37:24 CET 2023"                                                                
                                                                                                                   
  /* C compiler */                                                                                                 
  #define CC "gcc "                                                                                                
 @@ -85,7 +85,7 @@                                                                                                 
                                                                                                                   
  #define HAVE_ALPHASORT 1                                                                                         
                                                                                                                   
  /* Defined if you have the bigdft-atlab library. */                                                              
 -/* #undef HAVE_ATLAB */                                                                                          
 +#define HAVE_ATLAB 1                                                                                             
                                                                                                                   
  /* Defined if you have the BerkeleyGW library. */                                                                
  #define HAVE_BERKELEYGW 1                                                                                        
 @@ -127,7 +127,7 @@                                                                                               
                                                                                                                   
  /* #undef HAVE_CL_CL_H */                                                                                        
                                                                                                                   
  /* This is defined when we link with an external ISF library. */                                                 
 -/* #undef HAVE_COMP_ISF */                                                                                       
 +#define HAVE_COMP_ISF 1                                                                                          
                                                                                                                   
  /* This is defined when we link with the internal METIS library (default). */                                    
  /* #undef HAVE_COMP_METIS */                          

builder='foss2022a_serial_omp'                                           

                    left_url='https://www.octopus-code.org/buildbot/#/builders/31/builds/1024'                     

               right_url='http://mpsd-octopus-code-staging.desy.de/buildbot/#/builders/141/builds/7'

                                                                                                                   
 --- eb_foss2022a_serial_omp                                                                                       
                                                                                                                   
 +++ spack_foss2022a_serial_omp                                                                                    
                                                                                                                   
 @@ -2,7 +2,7 @@                                                                                                   
                                                                                                                   
  /* config.h.in.  Generated from configure.ac by autoheader.  */                                                  
                                                                                                                   
  /* date when configure was launched */                                                                           
 -#define BUILD_TIME "Mon Dec 11 22:10:54 CET 2023"                                                                
 +#define BUILD_TIME "Tue Dec 12 11:44:22 CET 2023"                                                                
                                                                                                                   
  /* C compiler */                                                                                                 
  #define CC "gcc "                                                                                                
 @@ -85,7 +85,7 @@                                                                                                 
                                                                                                                   
  #define HAVE_ALPHASORT 1                                                                                         
                                                                                                                   
  /* Defined if you have the bigdft-atlab library. */                                                              
 -/* #undef HAVE_ATLAB */                                                                                          
 +#define HAVE_ATLAB 1                                                                                             
                                                                                                                   
  /* Defined if you have the BerkeleyGW library. */                                                                
  #define HAVE_BERKELEYGW 1                                                                                        
 @@ -127,7 +127,7 @@                                                                                               
                                                                                                                   
  /* #undef HAVE_CL_CL_H */                                                                                        
                                                                                                                   
  /* This is defined when we link with an external ISF library. */                                                 
 -/* #undef HAVE_COMP_ISF */                                                                                       
 +#define HAVE_COMP_ISF 1                                                                                          
                                                                                                                   
  /* This is defined when we link with the internal METIS library (default). */                                    
  /* #undef HAVE_COMP_METIS */                          

builder='foss2022a_serial_debug'                                          

                    left_url='https://www.octopus-code.org/buildbot/#/builders/37/builds/1831'                     

              right_url='http://mpsd-octopus-code-staging.desy.de/buildbot/#/builders/152/builds/10'

                                                                                                                   
 --- eb_foss2022a_serial_debug                                                                                     
                                                                                                                   
 +++ spack_foss2022a_serial_debug                                                                                  
                                                                                                                   
 @@ -2,7 +2,7 @@                                                                                                   
                                                                                                                   
  /* config.h.in.  Generated from configure.ac by autoheader.  */                                                  
                                                                                                                   
  /* date when configure was launched */                                                                           
 -#define BUILD_TIME "Mon Dec 11 22:10:56 CET 2023"                                                                
 +#define BUILD_TIME "Tue Dec 12 11:39:43 CET 2023"                                                                
                                                                                                                   
  /* C compiler */                                                                                                 
  #define CC "gcc "                                                                                                
 @@ -85,7 +85,7 @@                                                                                                 
                                                                                                                   
  #define HAVE_ALPHASORT 1                                                                                         
                                                                                                                   
  /* Defined if you have the bigdft-atlab library. */                                                              
 -/* #undef HAVE_ATLAB */                                                                                          
 +#define HAVE_ATLAB 1                                                                                             
                                                                                                                   
  /* Defined if you have the BerkeleyGW library. */                                                                
  #define HAVE_BERKELEYGW 1                                                                                        
 @@ -127,7 +127,7 @@                                                                                               
                                                                                                                   
  /* #undef HAVE_CL_CL_H */                                                                                        
                                                                                                                   
  /* This is defined when we link with an external ISF library. */                                                 
 -/* #undef HAVE_COMP_ISF */                                                                                       
 +#define HAVE_COMP_ISF 1                                                                                          
                                                                                                                   
  /* This is defined when we link with the internal METIS library (default). */                                    
  /* #undef HAVE_COMP_METIS */                          